## View Raster Coordinate Reference System (CRS) in Python

You can view the `CRS` string associated with your `Python` object using the `crs()` method

In [1]:
# Import necessary packages
import os
import matplotlib.pyplot as plt
import rioxarray as rxr
import earthpy as et

In [3]:
# Get data and set working directory
et.data.get_data("colorado-flood")
os.chdir(os.path.join(et.io.HOME,
                     'earth-analytics',
                     'data'))

explore the metadata of a raster object using `rioxarray`. To begin, open up your data and view the CRS.

In [5]:
# Define relative path to file
lidar_dem_path = os.path.join("colorado-flood",
                             "spatial",
                             "boulder-leehill-rd",
                             "pre-flood",
                             "lidar",
                             "pre_DTM.tif")

# View crs of raster imported with rasterio
lidar_dem = rxr.open_rasterio(lidar_dem_path,masked=True)
print("The CRS of this data is:",lidar_dem.rio.crs)

The CRS of this data is: EPSG:32613


You can assign this string to a Python object, too. The example below only shows the code example to set a crs for an object where it is missing and you know what the CRS should be. In this case your data already has a defined CRS so this step is not necessary.

In [6]:
a_crs = lidar_dem.rio.crs
# Assign crs to myCRS object = this is just an example of how to do that
lidar_dem = lidar_dem.rio.set_crs(a_crs,inplace=True)

In [7]:
lidar_dem

<xarray.DataArray (band: 1, y: 2000, x: 4000)>
[8000000 values with dtype=float32]
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 4.72e+05 4.72e+05 4.72e+05 ... 4.76e+05 4.76e+05
  * y            (y) float64 4.436e+06 4.436e+06 ... 4.434e+06 4.434e+06
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

## Converting EPSG to Proj4 in Python

A python package for this class called ‘earthpy’ contains a dictionary that will help you convert EPSG codes into a Proj4 string. This can be used with rasterio in order to determine the metadata for a given EPSG code. For example, if you wish to know the units of the EPSG code above, you can do the following:

In [8]:
# Each key of the dictionary is an EPSG code
print(list(et.epsg.keys())[:10])

['29188', '26733', '24600', '32189', '4899', '29189', '26734', '7402', '26951', '29190']


You can convert to proj4 using `earthpy`:

In [9]:
# Convert to project string using earthpy
proj4 = et.epsg['32613']
print(proj4)

+proj=utm +zone=13 +datum=WGS84 +units=m +no_defs


You’ll focus on the first few components of the CRS in this tutorial.

1. +proj=utm The projection of the dataset. Your data are in Universal Transverse Mercator (UTM).
2. +zone=18 The UTM projection divides up the world into zones, this element tells you which zone the data is in. Harvard Forest is in Zone 18.
3. +datum=WGS84 The datum was used to define the center point of the projection. Your raster uses the WGS84 datum.
4. +units=m This is the horizontal units that the data are in. Your units are meters.

`Important: You are working with lidar data which has a Z or vertical value as well. While the horizontal units often match the vertical units of a raster they don’t always! Be sure the check the metadata of your data to figure out the vertical units!`

## Spatial Extent
The spatial extent of a raster or spatial object is the geographic area that the raster data covers.
The spatial extent of raster data. Notice that the spatial extent represents the rectangular area that the data cover. Thus, if the data are not rectangular (i.e. points OR an image that is rotated in some way) the spatial extent covers portions of the dataset where there are no data. Image Source: National Ecological Observatory Network (NEON).
https://www.earthdatascience.org/images/earth-analytics/spatial-data/spatial-extent.png

he spatial extent of an `Python` spatial object represents the geographic “edge” or location that is the furthest north, south, east and west. In other words, `extent` represents the overall geographic coverage of the spatial object.

You can access the spatial extent using the `.bounds()` attribute in `rasterio`.

In [10]:
lidar_dem.rio.bounds()

(472000.0, 4434000.0, 476000.0, 4436000.0)

## Raster Resolution

A raster has horizontal (x and y) resolution. This resolution represents the area on the ground that each pixel covers. The units for your data are in meters as determined by the CRS above. In this case, your data resolution is 1 x 1. This means that each pixel represents a 1 x 1 meter area on the ground. You can view the resolution of your data using the `.res` function.

In [11]:
# What is the x and y resolution for your raster data?
lidar_dem.rio.resolution()

(1.0, -1.0)